# Exact diagonalization study of 1/2 Heisenberg chain by Lanczos algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
# set parameters
N=6
dimension=2**N
#spin chain with all spin-down
z='0'*N
# initialize hamiltonian
H=np.zeros((dimension,dimension))
z

'000000'

In [ ]:
# Matrix Construction
for a in range(dimension):
    state_chain=bin(a)[2:] # the first two should be omitted for this 'bin' function
    l=len(state_chain)
    state_chain=z[0:N-l]+state_chain # make the length equal to N
    #print(state_chain)
  # for PBC, we set i in range(N)
  # for OBC, we set i in range(N-1)
    for i in range(N):
        j=np.mod(i+1,N)
        if state_chain[i]==state_chain[j]: # i=j only diagonal elements
            H[a,a]+=0.25
        else: # else, the raising/lowering operators also have contributions
            H[a,a]-=0.25
            # then exchange i,j
            element_i=state_chain[i]
            element_j=state_chain[j]
            #flip
            if j==0:
#here we are doing the concatenation of string (you can try other methods)
#                    print(state_chain)
                state_chain1=element_i+state_chain[1:N-1]+element_j
            else:
                state_chain1=state_chain[0:i]+element_j+element_i+state_chain[j+1:]
            b=int(state_chain1,2)
            H[a,b]+=0.5

# Lanczos algorithm

This note utilises the [Wikipedia on the algorithm](https://en.wikipedia.org/wiki/Lanczos_algorithm). Our Hamiltonian $H$ is a matrix of size $n\times n$, we want to simplify it to a tridiagonal matrix $T$ of size $m\times m$.

The steps are the given as:

1. for an arbitray complex vector $\nu_1=(\nu^1_1, \nu^2_1,\cdots,\nu^n_1)$ with norm $\langle \nu_1 |\nu_1\rangle=1$.
Let's do
$$
\begin{aligned}
\omega'_1 &= H \nu_1 \\
\alpha_1 &= \langle \omega'_1 | \nu_1\rangle \\
\omega_1 &= \omega'_1 - \alpha_1\nu_1
\end{aligned}
$$
<font color="red">(did you notice the Gram-Schmidt process?)
$$
\begin{aligned}
\nu_2 &= \frac{1}{\beta_2}(H\nu_1-\alpha_1\nu_1)\\
\langle \nu_2 | \nu_2 \rangle &= \frac{\langle \omega_1| \omega_1 \rangle}{\langle \omega_1 | \omega_1 \rangle}=1 \\
\langle \nu_1|\nu_2\rangle &= \frac{1}{\beta_2}(\langle \nu_1 | H|\nu_1\rangle-\langle\nu_1|\alpha_1|\nu_1\rangle) =0.
\end{aligned}
$$
</font>

2. for $j=2,\cdots,m$, do
      1. $\beta_j = \sqrt{\langle\omega_{j-1}|\omega_{j-1}\rangle}$
      2. $\nu_j = \omega_{j-1}/\beta_j$
      3. $\omega'_j = H\nu_j-\beta_j \nu\_{j-1}$
      4. $\alpha_j=\langle\omega'_j|\nu_j\rangle$
      5. $\omega_j=\omega'_j-\alpha_j \nu_j$

<font color="red">Such that
$$
\begin{aligned}
\nu_3 &= \frac{1}{\beta_3}(H\nu_{2}-\beta_{2}\nu_{1}-\alpha_{2}\nu_{2}) \\
&\cdots \\
\nu_j &= \frac{1}{\beta_j}(H\nu_{j-1}-\beta_{j-1}\nu_{j-2}-\alpha_{j-1}\nu_{j-1})\\
\langle \nu_{j-1}|\nu_j\rangle &= \frac{1}{\beta_j}(\langle \nu_{j-1} | H|\nu_{j-1}\rangle-\beta_{j-1}\langle \nu_{j-1}|\nu_{j-2}\rangle - \langle\nu_{j-1}|\alpha_{j-1}|\nu_{j-1}\rangle) \\
&=\frac{1}{\beta_j}(\langle \nu_{j-1}|H|\nu_{j-1}\rangle -\beta_{j-1}\langle\nu_{j-1}|\nu_{j-2}\rangle - \langle \nu_{j-1}|H|\nu_{j-1}\rangle - \beta_{j-1}\langle \nu_{j-1}|\nu_{j-2}\rangle)\\
&=0
\end{aligned}
$$
</font>

3. We generate the tridiagonal matrix $T$
$$
T= \begin{pmatrix}
\alpha_1 & \beta_2 & 0 & 0 & \cdots & 0 \\
\beta_2 & \alpha_2 & \beta_3 & 0 & \cdots & 0 \\
0 & \beta_3 & \alpha_3 & \beta_4  & \ddots & 0 \\
0 & 0 & \ddots & \ddots & \beta_{m-1} & 0 \\
0 & 0 & \cdots & \beta_{m-1} & \alpha_{m-1}& \beta_m \\
0 & 0 & \cdots & \cdots & \beta_m & \alpha_m
\end{pmatrix}
$$

One sees that a $n\times m$ matrix of $V=(\nu_1,\nu_2,\cdots,\nu_m)$ is formed with orthonormal columns and $H\nu_j = \beta_{j+1}\nu_{j+1}+\alpha_j\nu_j+\beta_j\nu_{j-1}$.

The relation between $n\times n$ Hamiltonian matrix and the $m\times m$ tridiagonal matrix $T$ is that
$$
T=V^{\dagger} H V.
$$

In [ ]:
def random_orthogonal(V,j):
    N = V.shape[0]
    v = np.random.randn(N)+1j*np.random.randn(N)
    for i in range(j):
        u = V[:,i]
        v -= np.vdot(u,v)*u/np.vdot(u,u)
    v = v/np.linalg.norm(v)
    return v
def Lanczos(Hamiltonian):
    H = np.copy(Hamiltonian)
    N = H.shape[0]
    m = min(100,N)
    H = 0.5*(H + H.conj().T)  # ensure Hermitian
    V_matrix = np.zeros((N, m), dtype=complex)
    T = np.zeros((m, m), dtype=complex)
    # init
    v = np.random.randn(N) + 1j*np.random.randn(N)
    v /= np.linalg.norm(v)
    V_matrix[:, 0] = v

    w_prime = H @ v
    alpha = np.vdot(v, w_prime)
    w = w_prime - alpha * v
    T[0, 0] = alpha

    for i in range(1, m):
        if (i+1)%(m//10) == 0:
            percent = int((i+1)/m*100)
            print(f"Lanczos Progress:{percent}%")
#        h = V_matrix[:, :i].conj().T @ w
#        w -= V_matrix[:, :i] @ h
        beta = np.linalg.norm(w)
        v_old = v
        if beta < 10e-6:
            v = random_orthogonal(V_matrix,i)
        else:
            v = w / beta
        V_matrix[:, i] = v
        w_prime = H @ v - beta * v_old
        alpha = np.vdot(v, w_prime)
        w = w_prime - alpha * v

        T[i, i] = alpha
        T[i, i-1] = beta
        T[i-1, i] = beta

    return T




# Implicitly Shifted QR Decomposition Method to solve the eigenvalues of tridiagonal matrix

Algorithm 4 from the book: [Handbook of Linear Algebra by Hogben](https://math.ecnu.edu.cn/~jypan/Teaching/MC/refs/2014%20Symmetric%20Matrix%20Eigenvalue%20Techniques.pdf)




In [ ]:
def Givens(x,y):
    G = np.zeros([2,2])
    if y == 0:
        c = 1
        s = 0
    elif x == 0:
        c = 0
        s = np.sign(y)
    else:
        c = abs(x)/math.sqrt(abs(x)**2+abs(y)**2)
        s = np.sign(x)*y/math.sqrt(abs(x)**2+abs(y)**2)
    G[0,0] = c
    G[1,1] = c
    G[0,1] = s
    G[1,0] = -s
    return G

def QR_iteration(T_matrix):
    T = np.copy(T_matrix)
    T = np.real(T)
    epsilon = 10e-6
    if T.shape == (1,1):
        return T
    n_itr = 0

    while True:
        n_itr += 1
        N = T.shape[0]

        tau = (T[N-2,N-2]-T[N-1,N-1])/2
        sgn = np.copysign(1.0,tau)
        den = tau + sgn * np.sqrt(tau*tau+T[N-1,N-2]**2)
        if den == 0:
            mu = T[N-1,N-1]
        else:
            mu = T[N-1,N-1]-T[N-1,N-2]**2/den

        for i in range(0,N-1):
            if i==0:
                G = Givens(T[0,0]-mu,T[1,0])
            else:
                G = Givens(T[i,i-1],T[i+1,i-1])

            iL = max(i-1,0)
            iR = min(N,i+3)

            T[i:i+2,iL:iR] = G@T[i:i+2,iL:iR]
            T[iL:iR,i:i+2] = T[iL:iR,i:i+2]@G.T

            if abs(T[i,i+1])**2 <= epsilon**2*abs(T[i,i]*T[i+1,i+1]):
                #print(n_itr)
                T[i,i+1] = T[i+1,i] = 0.0
                T[0:i+1,0:i+1] = QR_iteration(T[0:i+1,0:i+1])
                T[i+1:N,i+1:N] = QR_iteration(T[i+1:N,i+1:N])
                return T



In [ ]:
T = Lanczos(H)
Lambda = QR_iteration(T)
eigs_lan = np.diag(Lambda)
eigs_lan = np.sort(eigs_lan)
print(eigs_lan)

Lanczos Progress:9%
Lanczos Progress:18%
Lanczos Progress:28%
Lanczos Progress:37%
Lanczos Progress:46%
Lanczos Progress:56%
Lanczos Progress:65%
Lanczos Progress:75%
Lanczos Progress:84%
Lanczos Progress:93%
[-2.80277564e+00 -2.80277564e+00 -2.80277563e+00 -2.69722061e+00
 -2.11803399e+00 -2.11803399e+00 -2.11803399e+00 -2.11803379e+00
 -2.11803376e+00 -1.50000000e+00 -1.50000000e+00 -1.30152318e+00
 -1.28077641e+00 -1.28077641e+00 -1.28077641e+00 -1.28077641e+00
 -1.28077641e+00 -1.00000007e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -9.80562982e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -4.99999999e-01
 -3.93991004e-01  4.24783254e-15  1.86464634e-13  5.82136720e-12
  1.39081610e-10  1.12990101e-08  2.58448262e-02  1.18033986e-01
  1.18033989e-01  1.18033989e-01  1.18033989e-01  1.18033990e-01
  4.99999854e-01  5.00000000e-01  5.00000000e-01  5.00000000e-01
  5.00005898e-01  7.80776406e-01  7.80776406e-01  7.80776406e-01
  7.8077640

In [ ]:
eig_value_H=np.real(np.linalg.eig(H)[0])
eig_value_H = np.sort(eig_value_H)
print(eig_value_H)

[-2.80277564e+00 -2.11803399e+00 -2.11803399e+00 -2.11803399e+00
 -1.50000000e+00 -1.28077641e+00 -1.28077641e+00 -1.28077641e+00
 -1.28077641e+00 -1.28077641e+00 -1.28077641e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00
 -1.00000000e+00 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -4.27008312e-16 -2.07469137e-16 -3.84625944e-17 -3.84625944e-17
 -3.61767163e-17 -8.67861560e-19  1.06119349e-17  8.09655814e-17
  9.18770360e-17  1.62788760e-16  1.18033989e-01  1.18033989e-01
  1.18033989e-01  5.00000000e-01  5.00000000e-01  5.00000000e-01
  7.80776406e-01  7.80776406e-01  7.80776406e-01  7.80776406e-01
  7.80776406e-01  7.80776406e-01  8.02775638e-01  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.50000000e+00  1.50000000e+00  1.50000000e+00
  1.50000000e+00  1.50000